In [11]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable
import csv
from subprocess import check_output
from time import sleep

## read the data

In [12]:
train = pd.read_csv("../ML_final/train.csv")
test = pd.read_csv("../ML_final/test.csv")

In [13]:
train_x = train.drop(['id'], axis=1).drop(['product_code'], axis=1).drop(['failure'], axis=1)
train_x = train_x.to_numpy()
train_y = train[['failure']]
train_y = train_y.to_numpy()
print(train_x[0])
for i in range(train_x.shape[0]):
    train_x[i][1] = train_x[i][1].split('_')[-1]
    train_x[i][2] = train_x[i][2].split('_')[-1]


train_x = train_x.astype(np.float32)
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

train_imp = imputer.fit(train_x)
train_x = train_imp.transform(train_x)
train_y = train_y.astype(np.float32)

test_x = test.drop(['id'], axis=1).drop(['product_code'], axis=1)
test_x = test_x.to_numpy()
print(train_x[0])
for i in range(test_x.shape[0]):
    test_x[i][1] = test_x[i][1].split('_')[-1]
    test_x[i][2] = test_x[i][2].split('_')[-1]

test_x = test_x.astype(np.float32)
test_imp = imputer.fit(test_x)
test_x = train_imp.transform(test_x)

[80.1 'material_7' 'material_8' 9 5 7 8 4 18.04 12.518 15.748 19.292
 11.739 20.155 10.672 15.859 17.594 15.193 15.029 nan 13.034 14.684 764.1]
[ 80.1     7.      8.      9.      5.      7.      8.      4.     18.04
  12.518  15.748  19.292  11.739  20.155  10.672  15.859  17.594  15.193
  15.029  16.04   13.034  14.684 764.1  ]


## define NN model

In [14]:
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation
# model = Sequential()
#     # 加入 hidden layer-1 of 512 neurons 並指定 input_dim 為 784
# model.add(Dense(92, input_dim=23))
# # 使用 'relu' 當作 activation function
# model.add(Activation('relu'))
# # 使用 Dropout 隨機捨棄 30% 的連線
# model.add(Dropout(0.3))
# # 加入 hidden layer-2 of 256 neurons
# model.add(Dense(92))
# # 使用 'relu' 當作 activation function
# model.add(Activation('relu'))
# # 使用 Dropout 隨機捨棄 30% 的連線
# model.add(Dropout(0.3))
# # 加入 hidden layer-3 of 128 neurons
# model.add(Dense(1))
# # 使用 'relu' 當作 activation function
# model.add(Activation('sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# train_history = model.fit(x=train_x, y=train_y, validation_split=0.1,  epochs=200, batch_size=200, verbose=2)

In [16]:
best_score = 0.0
for i in range(1000):    
    model = Sequential()
    # 加入 hidden layer-1 of 512 neurons 並指定 input_dim 為 784
    model.add(Dense(92, input_dim=23))
    # 使用 'relu' 當作 activation function
    model.add(Activation('relu'))
    # 使用 Dropout 隨機捨棄 30% 的連線
    model.add(Dropout(0.3))
    # 加入 hidden layer-2 of 256 neurons
    model.add(Dense(92))
    # 使用 'relu' 當作 activation function
    model.add(Activation('relu'))
    # 使用 Dropout 隨機捨棄 30% 的連線
    model.add(Dropout(0.3))
    # 加入 hidden layer-3 of 128 neurons
    model.add(Dense(1))
    # 使用 'relu' 當作 activation function
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    train_history = model.fit(x=train_x, y=train_y, validation_split=0.1,  epochs=200, batch_size=200, verbose=2)
    y_pred = model.predict(x=test_x, batch_size=200)
    answer = np.array([])
    for pred in y_pred:
        pred = pred.ravel()
        answer = np.append(answer, pred)

    submission = pd.read_csv("../ML_final/sample_submission.csv")
    submission["failure"] = answer
    submission.reset_index(drop=True).to_csv("submission_tensorflow.csv", index=False)
    check_output('kaggle competitions submit -c tabular-playground-series-aug-2022 -f submission_tensorflow.csv -m ' + str(i), shell=True)
    sleep(10)
    result = check_output(f'kaggle competitions submissions -c tabular-playground-series-aug-2022', shell=True).decode()
    x = result.find('complete')
    score = float(result[x+23:x+30])
    print (score)
    
    if(best_score < score):

        model.save(str(score) + '.h5')
        best_score = score
        print(score)

Epoch 1/200
120/120 - 1s - loss: 7.1450 - accuracy: 0.6646 - val_loss: 0.5228 - val_accuracy: 0.7922 - 572ms/epoch - 5ms/step
Epoch 2/200
120/120 - 0s - loss: 0.9132 - accuracy: 0.7223 - val_loss: 0.5365 - val_accuracy: 0.7922 - 164ms/epoch - 1ms/step
Epoch 3/200
120/120 - 0s - loss: 0.6174 - accuracy: 0.7675 - val_loss: 0.5327 - val_accuracy: 0.7922 - 175ms/epoch - 1ms/step
Epoch 4/200
120/120 - 0s - loss: 0.5760 - accuracy: 0.7774 - val_loss: 0.5224 - val_accuracy: 0.7922 - 168ms/epoch - 1ms/step
Epoch 5/200
120/120 - 0s - loss: 0.5562 - accuracy: 0.7819 - val_loss: 0.5193 - val_accuracy: 0.7922 - 183ms/epoch - 2ms/step
Epoch 6/200
120/120 - 0s - loss: 0.5445 - accuracy: 0.7836 - val_loss: 0.5192 - val_accuracy: 0.7922 - 178ms/epoch - 1ms/step
Epoch 7/200
120/120 - 0s - loss: 0.5382 - accuracy: 0.7846 - val_loss: 0.5141 - val_accuracy: 0.7922 - 189ms/epoch - 2ms/step
Epoch 8/200
120/120 - 0s - loss: 0.5370 - accuracy: 0.7851 - val_loss: 0.5215 - val_accuracy: 0.7922 - 182ms/epoch - 2

100%|██████████| 520k/520k [04:24<00:00, 2.02kB/s]  


0.59025
0.59025
Epoch 1/200
120/120 - 1s - loss: 7.6585 - accuracy: 0.6625 - val_loss: 0.5904 - val_accuracy: 0.7922 - 535ms/epoch - 4ms/step
Epoch 2/200
120/120 - 0s - loss: 1.1301 - accuracy: 0.7020 - val_loss: 0.5626 - val_accuracy: 0.7922 - 164ms/epoch - 1ms/step
Epoch 3/200
120/120 - 0s - loss: 0.6191 - accuracy: 0.7646 - val_loss: 0.5190 - val_accuracy: 0.7922 - 168ms/epoch - 1ms/step
Epoch 4/200
120/120 - 0s - loss: 0.5609 - accuracy: 0.7792 - val_loss: 0.5124 - val_accuracy: 0.7922 - 157ms/epoch - 1ms/step
Epoch 5/200
120/120 - 0s - loss: 0.5485 - accuracy: 0.7830 - val_loss: 0.5144 - val_accuracy: 0.7922 - 159ms/epoch - 1ms/step
Epoch 6/200
120/120 - 0s - loss: 0.5417 - accuracy: 0.7841 - val_loss: 0.5122 - val_accuracy: 0.7922 - 159ms/epoch - 1ms/step
Epoch 7/200
120/120 - 0s - loss: 0.5359 - accuracy: 0.7846 - val_loss: 0.5176 - val_accuracy: 0.7922 - 157ms/epoch - 1ms/step
Epoch 8/200
120/120 - 0s - loss: 0.5327 - accuracy: 0.7853 - val_loss: 0.5097 - val_accuracy: 0.7922 -

100%|██████████| 520k/520k [06:35<00:00, 1.35kB/s]  


0.58999
Epoch 1/200
120/120 - 1s - loss: 7.8306 - accuracy: 0.6619 - val_loss: 0.6927 - val_accuracy: 0.4765 - 564ms/epoch - 5ms/step
Epoch 2/200
120/120 - 0s - loss: 0.8931 - accuracy: 0.7294 - val_loss: 0.5393 - val_accuracy: 0.7922 - 170ms/epoch - 1ms/step
Epoch 3/200
120/120 - 0s - loss: 0.6013 - accuracy: 0.7730 - val_loss: 0.5177 - val_accuracy: 0.7922 - 162ms/epoch - 1ms/step
Epoch 4/200
120/120 - 0s - loss: 0.5618 - accuracy: 0.7815 - val_loss: 0.5273 - val_accuracy: 0.7922 - 161ms/epoch - 1ms/step
Epoch 5/200
120/120 - 0s - loss: 0.5512 - accuracy: 0.7834 - val_loss: 0.5284 - val_accuracy: 0.7922 - 160ms/epoch - 1ms/step
Epoch 6/200
120/120 - 0s - loss: 0.5421 - accuracy: 0.7842 - val_loss: 0.5179 - val_accuracy: 0.7922 - 156ms/epoch - 1ms/step
Epoch 7/200
120/120 - 0s - loss: 0.5360 - accuracy: 0.7852 - val_loss: 0.5216 - val_accuracy: 0.7922 - 171ms/epoch - 1ms/step
Epoch 8/200
120/120 - 0s - loss: 0.5348 - accuracy: 0.7858 - val_loss: 0.5174 - val_accuracy: 0.7922 - 174ms/e

...

0.58993
Epoch 1/200
120/120 - 1s - loss: 8.2270 - accuracy: 0.6586 - val_loss: 0.5768 - val_accuracy: 0.7915 - 570ms/epoch - 5ms/step
Epoch 2/200
120/120 - 0s - loss: 1.0572 - accuracy: 0.7070 - val_loss: 0.5482 - val_accuracy: 0.7922 - 174ms/epoch - 1ms/step
Epoch 3/200
120/120 - 0s - loss: 0.6104 - accuracy: 0.7669 - val_loss: 0.5342 - val_accuracy: 0.7922 - 165ms/epoch - 1ms/step
Epoch 4/200
120/120 - 0s - loss: 0.5690 - accuracy: 0.7792 - val_loss: 0.5278 - val_accuracy: 0.7922 - 157ms/epoch - 1ms/step
Epoch 5/200
120/120 - 0s - loss: 0.5483 - accuracy: 0.7828 - val_loss: 0.5264 - val_accuracy: 0.7922 - 152ms/epoch - 1ms/step
Epoch 6/200
120/120 - 0s - loss: 0.5436 - accuracy: 0.7843 - val_loss: 0.5209 - val_accuracy: 0.7922 - 156ms/epoch - 1ms/step
Epoch 7/200
120/120 - 0s - loss: 0.5390 - accuracy: 0.7847 - val_loss: 0.5313 - val_accuracy: 0.7922 - 170ms/epoch - 1ms/step
Epoch 8/200
120/120 - 0s - loss: 0.5353 - accuracy: 0.7857 - val_loss: 0.5152 - val_accuracy: 0.7922 - 161ms/e

100%|██████████| 521k/521k [00:03<00:00, 170kB/s]


0.5896
Epoch 1/200
120/120 - 1s - loss: 6.5975 - accuracy: 0.6660 - val_loss: 0.5401 - val_accuracy: 0.7922 - 515ms/epoch - 4ms/step
Epoch 2/200
120/120 - 0s - loss: 0.8374 - accuracy: 0.7296 - val_loss: 0.5351 - val_accuracy: 0.7922 - 146ms/epoch - 1ms/step
Epoch 3/200
120/120 - 0s - loss: 0.5997 - accuracy: 0.7756 - val_loss: 0.5259 - val_accuracy: 0.7922 - 149ms/epoch - 1ms/step
Epoch 4/200
120/120 - 0s - loss: 0.5663 - accuracy: 0.7815 - val_loss: 0.5190 - val_accuracy: 0.7922 - 148ms/epoch - 1ms/step
Epoch 5/200
120/120 - 0s - loss: 0.5521 - accuracy: 0.7832 - val_loss: 0.5251 - val_accuracy: 0.7922 - 147ms/epoch - 1ms/step
Epoch 6/200
120/120 - 0s - loss: 0.5439 - accuracy: 0.7847 - val_loss: 0.5192 - val_accuracy: 0.7922 - 147ms/epoch - 1ms/step
Epoch 7/200
120/120 - 0s - loss: 0.5380 - accuracy: 0.7853 - val_loss: 0.5180 - val_accuracy: 0.7922 - 147ms/epoch - 1ms/step
Epoch 8/200
120/120 - 0s - loss: 0.5342 - accuracy: 0.7858 - val_loss: 0.5151 - val_accuracy: 0.7922 - 157ms/ep

100%|██████████| 520k/520k [00:02<00:00, 190kB/s]


0.59042
0.59042
Epoch 1/200
120/120 - 1s - loss: 10.8079 - accuracy: 0.6552 - val_loss: 2.3295 - val_accuracy: 0.7922 - 524ms/epoch - 4ms/step
Epoch 2/200
120/120 - 0s - loss: 2.3917 - accuracy: 0.6637 - val_loss: 0.5580 - val_accuracy: 0.7922 - 151ms/epoch - 1ms/step
Epoch 3/200
120/120 - 0s - loss: 0.7747 - accuracy: 0.7255 - val_loss: 0.5391 - val_accuracy: 0.7922 - 151ms/epoch - 1ms/step
Epoch 4/200
120/120 - 0s - loss: 0.5967 - accuracy: 0.7636 - val_loss: 0.5197 - val_accuracy: 0.7922 - 150ms/epoch - 1ms/step
Epoch 5/200
120/120 - 0s - loss: 0.5636 - accuracy: 0.7756 - val_loss: 0.5124 - val_accuracy: 0.7922 - 152ms/epoch - 1ms/step
Epoch 6/200
120/120 - 0s - loss: 0.5476 - accuracy: 0.7809 - val_loss: 0.5154 - val_accuracy: 0.7922 - 150ms/epoch - 1ms/step
Epoch 7/200
120/120 - 0s - loss: 0.5411 - accuracy: 0.7831 - val_loss: 0.5179 - val_accuracy: 0.7922 - 167ms/epoch - 1ms/step
Epoch 8/200
120/120 - 0s - loss: 0.5369 - accuracy: 0.7833 - val_loss: 0.5111 - val_accuracy: 0.7922 

In [ ]:
# model.save('final.h5')